In [2]:
from pathlib import Path
import time
import requests
from urllib.parse import urljoin
import bs4
import pymongo
from datetime import datetime


def get_response( url, *args, **kwargs):
    for _ in range(15):  
        response = requests.get(url, *args, **kwargs)
        if response.status_code == 200:
            return response
        time.sleep(1)
    raise ValueError("URL DIE")

def get_soup(url, *args, **kwargs) -> bs4.BeautifulSoup:
    soup = bs4.BeautifulSoup(get_response(url, *args, **kwargs).text, "lxml")
    return soup

def price(a,clas):
    try:
        if a.find('div', attrs={'class': clas}):
            old = a.find('div', attrs={'class': clas})
            price_int = old.find("span", class_='label__price-integer')
            price_deci = old.find("span", class_='label__price-decimal')
            return (f'{price_int.text},{price_deci.text}')
        else:
            return None
    except AttributeError:
        pass       
    
def time(a,date):
    month = {'января':"01",'февраля':"02",'марта':"03","апреля":"04","мая":"05","июня":"06","июля":"07","августа":"08",
    "сентября":"09","октября":"10","ноября":"11","декабря":"12"}
    aux = {"date_from":{'start':1,'stop':2},
      "date_to":{'start':4,'stop':5}}
    if  a.find('div', attrs={'class': "card-sale__date"}):
        time = a.find('div', attrs={'class': "card-sale__date"})
        date_day = (time.text).split()[aux[date]['start']]  
        date_month = month[(time.text).split()[aux[date]['stop']]]
        datem = datetime.today().strftime("%Y")
        date = (f'{date_day}/{date_month}/{datem}')
        date_dt2 = datetime.strptime(date, '%d/%m/%Y')
        return date_dt2
    else:
        return None

def head(a):
    if a.find("div", attrs={"class": "card-sale__header"}):
        return a.find("div", attrs={"class": "card-sale__header"}).text
    else:
        return None
    
def save(data):
        collection = db["magnit"]
        collection.insert_one(data)

def urll(a):
    return urljoin(url, a.attrs.get("href", "/"))

def product_na(a):
    if  a.find("div", attrs={"class": "card-sale__title"}):
        return  a.find("div", attrs={"class": "card-sale__title"}).text
    else:
        return None

def image(a):
    return urljoin(url, a.find("picture").find("img").attrs.get("data-src", "/"))

def _parse(a):
    product_data = {
    "url": urll(a),
    "promo_name": head(a),
    "product_name": product_na(a),
    "old_price": price(a,'label__price label__price_old'),
    "new_price": price(a,'label__price label__price_new'),
    "image_url": image(a),
    "date_from": time(a,'date_from'),
    "date_to": time(a,'date_to')
    }
    return product_data



if __name__ == "__main__":

    url = "https://magnit.ru/promo/"
    mongo_url = "mongodb://localhost:27017"
    client = pymongo.MongoClient(mongo_url)
    db = client["gb_parse_28_03_21_3"]  

    products_a = get_soup(url).find_all("a", attrs={"class": "card-sale"})

    for a in products_a:
        save(_parse(a))
